## Accumaulator

모든 Executor가 함께 쓸수 있는 쓰기 전용 전역 변수

1. 변수를 생성하고 작업이 끝난 후 결과(value)를 읽을 수 있다.
2. 작업을 처리하면서 값을 더할 수는 있지만, 내부의 값을 읽을 수는 없다.

>꼭 ! Action(foreach,count)등에서만 사용해야 한다
>
>map,filter에서 사용시 카운트가 중복될 수  있다



| **구분**     | **Accumulator**                  | **filter().count()**               |
| ---------- | -------------------------------- | ---------------------------------- |
| **장점**     | **빠름:** 데이터 한 번 훑을 때 덤으로 셈.      | **안전함:** 에러 확률 0%, 코드가 직관적임.       |
| **단점**     | **위험함:** 중복 집계 가능성 있음.           | **느림:** 데이터를 한 번 더 읽어야 함 (I/O 발생). |
| **용도**     | 단순 로그, 디버깅, 모니터링                 | 결제 금액 합산, 정산, 핵심 지표                |
| **코드 스타일** | `foreach` 안에서 `acc.add(1)` (절차적) | `filter(...)` -> `count()` (함수형)   |
| **실행 흐름**  | 데이터를 한 번 훑으면서 옆에다 기록함.           | 데이터를 걸러내는 **새로운 RDD**를 만들고 숫자를 셈.  |
| **안전성**    | 재수 없으면 중복 카운트 될 수 있음.            | **100% 정확함.** (스파크가 보장함)           |


In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [2]:
spark=(
    SparkSession
    .builder
    .appName("accumaulator")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 14:12:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
sc=spark.sparkContext

In [8]:
# 짝수 개수 보기

In [6]:
rdd=sc.range(1,101)

even_acc=sc.accumulator(0)

def count_even(x):
    global even_acc
    if x%2==0:
        even_acc.add(1)

rdd.foreach(count_even)
print(f"짝수개수는 {even_acc.value}")

짝수개수는 50


In [9]:
# 불량데이터 가 몇개가 있나 확인 

In [16]:
raw_data = ["sensor_1", "sensor_2", None, "sensor_3", "", "sensor_4", None]
rdd=sc.parallelize(raw_data)

error_acc=sc.accumulator(0)

def process_sensor(data):
    global error_acc
    if data is None or data =='':
        error_acc.add(1)
        return
    else:
        print(f"정상데이터이므로 처리했다. {data}")

rdd.foreach(process_sensor)
print(f"발견돈 불량데이터수 :{error_acc.value}")

발견돈 불량데이터수 :3


In [17]:
# 정산 시스템 오류

In [19]:
transactions = [1000, -500, 2000, 0, 3000, -100, 5000]
rdd=sc.parallelize(transactions)

error_txs=rdd.filter(lambda x: x<=0)
error_count=error_txs.count()
print(f"발견된 오류 개수:{error_count}")

발견된 오류 개수:3
